<a href="https://colab.research.google.com/github/goya5858/OSERO/blob/main/DQN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import torch.multiprocessing as mp
import time

import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gym
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import functional as F

BATCH_SIZE=32

# Agent

In [2]:
class Networks(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(in_features=4, out_features=100)
        self.act1 = nn.ReLU()
        #self.drop1 = nn.Dropout(p=0.3)

        self.linear2 = nn.Linear(in_features=100, out_features=100)
        self.act2 = nn.ReLU()
        #self.drop2 = nn.Dropout(p=0.3)

        self.Q_network = nn.Linear(in_features=100, out_features=2)
        #self.pi_softmax = nn.Softmax()
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        #x = self.drop1(x)

        x = self.linear2(x)
        x = self.act2(x)
        #x = self.drop2(x)

        Q_value = self.Q_network(x)
        return Q_value

In [36]:
class Agetnt_Memory():
    def __init__(self):
        self.States   = []
        self.Actions = []
        self.Rewords = []
        self.Next_States = []
    
    def put_memory(self, state, action, reword, next_state):
        self.States.append(state)
        self.Actions.append(action)
        self.Rewords.append(reword)
        self.Next_States.append(next_state)

        if len(self.Actions) > 5000:
            self.States   = self.States[:-5000]
            self.Actions = self.Actions[:-5000]
            self.Rewords = self.Rewords[:-5000]
            self.Next_States = self.Next_States[:-5000]

    def get_batch(self, BATCH_SIZE):
        num_memories = len(self.States)
        sample_index = random.sample( range(num_memories), min([BATCH_SIZE, num_memories]) )

        print(np.stack( np.array(self.States)[sample_index]  ))
        States_BATCH = torch.tensor( np.stack( np.array(self.States)[sample_index]  ) ).to(device)
        Action_BATCH = torch.tensor( np.stack( np.array(self.Actions)[sample_index] ) ).to(device).reshape(-1,1)
        Reword_BATCH = torch.tensor( np.stack( np.array(self.Rewords)[sample_index] ) ).to(device)
        #Next_States_BATCH = torch.tensor( np.stack( np.array(self.Next_States)[sample_index] ) ).to(device)
        Next_States_BATCH = np.array(self.Next_States)[sample_index]
        Next_S_non_mask = [s is not None for s in Next_States_BATCH]
        Next_States_BATCH = torch.tensor( np.stack(Next_States_BATCH[Next_S_non_mask]) ).to(device)
        return States_BATCH, \
               Action_BATCH, \
               Reword_BATCH, \
               Next_States_BATCH, \
               Next_S_non_mask

In [32]:
class Agent:
    # 実際に行動を行うAgent
    # DQNで学習する
    def __init__(self):
        self.gamma = torch.tensor(0.99).to(device)
        self.episode = 0

        self.Memory = Agetnt_Memory()

        self.Network = Networks().double().to(device)
        self.Network_target = Networks().double().to(device)
        self.Network_target.load_state_dict( self.Network.state_dict() )

        torch.nn.utils.clip_grad_norm_(self.Network.parameters(), 0.1)
        self.optim = torch.optim.Adam(params=self.Network.parameters(), lr=0.0005 )
    
    def get_action(self, state):
        #ε-greedy法で行動を決定します
        #stateを入力として受け取り、actionを出力として返す
        #if self.episode >= 500:   
        #    eps = 0.0
        #else:
        #    eps = 0.3 * (1 - self.episode/500)  # linearly interpolate

        #if random.random() < eps:
        if 0 > 1:
            #action = random.randint(0, 2 - 1)   # ランダムに行動
            pass
        else:
            Q_values = self.Network( torch.tensor(state).double().to(device) )
            p_values = nn.Softmax()( Q_values )
            action = np.random.choice(2, p=p_values.cpu().detach().numpy()[0])
        return action

    def put_memory(self, state, action, reword, next_state):
        self.Memory.put_memory(state, action, reword, next_state)

    def network_update(self, BATCH_SIZE=BATCH_SIZE):
        # AgentのMemoriesからBATCH_SIZEの分だけデータをランダムに取り出す
        States_BATCH, Action_BATCH, Reword_BATCH, Next_States_BATCH, Next_S_non_mask = self.Memory.get_batch(BATCH_SIZE)

        Q_values = self.Network( States_BATCH )
        Next_Q_values = torch.zeros_like( Q_values )

        Q_values = Q_values.gather( -1, Action_BATCH ) #Gatherメソッドでdim=1方向でそれぞれの行でスライスできる

        Next_Q_values[Next_S_non_mask] = self.Network_target( Next_States_BATCH )
        Next_Q_values = Next_Q_values.max(dim=1).values

        True_Values = Reword_BATCH + self.gamma * Next_Q_values
        loss = nn.MSELoss()(Q_values, True_Values)
    
        self.optim.zero_grad()
        loss.backward()
        self.optim.step()
        #print('loss :', loss.cpu().detach())
    
    def reset_model(self, episode):
        self.episode = episode
        self.Network_target.load_state_dict( self.Network.state_dict() )

In [16]:
# namedtupleを生成
from collections import namedtuple

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

# 経験を保存するメモリクラスを定義します
class ReplayMemory:
    def __init__(self, CAPACITY):
        self.capacity = CAPACITY  # メモリの最大長さ
        self.memory = []  # 経験を保存する変数
        self.index = 0  # 保存するindexを示す変数

    def push(self, state, action, state_next, reward):
        '''transition = (state, action, state_next, reward)をメモリに保存する'''

        if len(self.memory) < self.capacity:
            self.memory.append(None)  # メモリが満タンでないときは足す

        self.memory[self.index] = Transition(state, action, state_next, reward)

        self.index = (self.index + 1) % self.capacity  # 保存するindexを1つずらす

    def sample(self, batch_size):
        '''batch_size分だけ、ランダムに保存内容を取り出す'''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''関数lenに対して、現在の変数memoryの長さを返す'''
        return len(self.memory)

# エージェントが持つ脳となるクラスです、DQNを実行します
# Q関数をディープラーニングのネットワークをクラスとして定義

import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000


class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)  # ネットワークの形を出力

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.1)

    def replay(self):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        if len(self.memory) < BATCH_SIZE:
            return

        transitions = self.memory.sample(BATCH_SIZE)
        batch = Transition(*zip(*transitions))

        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])

        state_action_values = self.model(state_batch).gather(1, action_batch)

        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))
        # まずは全部0にしておく
        next_state_values = torch.zeros(BATCH_SIZE)
        next_state_values[non_final_mask] = self.model(non_final_next_states).max(1)[0].detach()

        expected_state_action_values = reward_batch + GAMMA * next_state_values

        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

        # 4.3 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optimizer.step()  # 結合パラメータを更新

    def decide_action(self, state, episode):
        '''現在の状態に応じて、行動を決定する'''
        # ε-greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値のindexを取り出します = max(1)[1]
            # .view(1,1)は[torch.LongTensor of size 1]　を size 1x1 に変換します

        else:
            # 0,1の行動をランダムに返す
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 0,1の行動をランダムに返す
            # actionは[torch.LongTensor of size 1x1]の形になります

        return action

class Agent:
    def __init__(self, num_states, num_actions):
        '''課題の状態と行動の数を設定する'''
        self.brain = Brain(num_states, num_actions)  # エージェントが行動を決定するための頭脳を生成

    def update_q_function(self):
        '''Q関数を更新する'''
        self.brain.replay()

    def get_action(self, state, episode):
        '''行動を決定する'''
        action = self.brain.decide_action(state, episode)
        return action

    def memorize(self, state, action, state_next, reward):
        '''memoryオブジェクトに、state, action, state_next, rewardの内容を保存する'''
        self.brain.memory.push(state, action, state_next, reward)

In [200]:
from collections import namedtuple
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'state_next'))

class Agent_Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1  = nn.Linear(4, 32)
        self.act1 = nn.ReLU()

        self.fc2  = nn.Linear(32, 32)
        self.act2 = nn.ReLU()

        self.fc3  = nn.Linear(32, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)

        x = self.fc2(x)
        x = self.act2(x)

        x = self.fc3(x)
        return x

class Agent_Memory:
    def __init__(self):
        self.capacity = CAPACITY #メモリの大きさ
        self.memory = []
        self.index = 0

    def sample(self, BATCH_SIZE):
        '''batch_size分だけ、ランダムに保存内容を取り出す'''
        if len(self.memory) < BATCH_SIZE:
            BATCH_SIZE = len(self.memory)

        transitions = random.sample(self.memory, BATCH_SIZE) #BATCH_SIZEの分だけランダムにデータを取り出す
        BATCH = Transition(*zip(*transitions)) #State, Action, Reward, State_Nextの各要素ごとにまとめ直す

        State_BATCH      = torch.tensor( np.stack( BATCH.state ) ,dtype=torch.float)
        Action_BATCH     = torch.tensor( np.stack( BATCH.action ) ,dtype=torch.int64)
        Reward_BATCH     = torch.tensor( np.stack( BATCH.reward ) ,dtype=torch.float)
        State_Next_BATCH = np.array( BATCH.state_next ) 
        non_final_mask   = [s is not None for s in State_Next_BATCH]
        State_Next_BATCH = torch.tensor( np.stack(State_Next_BATCH[non_final_mask] ) ,dtype=torch.float)
        return State_BATCH, Action_BATCH, Reward_BATCH, State_Next_BATCH, non_final_mask



    def put_memory(self, state, action, reward, state_next):
        '''transition = (state, action, reward, state_next)をメモリに保存する'''
        if len(self.memory) < self.capacity:
            self.memory.append(None)  # メモリが満タンでないときは足す
        self.memory[self.index] = Transition(state, action, reward, state_next)
        self.index = (self.index + 1) % self.capacity  # 保存するindexを1つずらす

class Agent:
    def __init__(self):
        self.Network = Agent_Network()
        self.Memory  = Agent_Memory()
        self.optim   = optim.Adam(self.Network.parameters(), lr=0.0001)

    def get_action(self, state, episode):
        state = torch.tensor( state, dtype=torch.float )
        # ε-greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.Network.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                Q_values = self.Network( state )
                action = Q_values.argmax().detach().numpy()
                print('OK')
        else:
            # 0,1の行動をランダムに返す
            action = np.random.choice(2)   # 0,1の行動をランダムに返す
            action = np.array(action)
        return action

    def update_Network(self, BATCH_SIZE):
        if len(self.Memory.memory) < BATCH_SIZE:
            BATCH_SIZE = len(self.Memory.memory)

        # Memoryから学習用のデータをランダムに取り出す
        State_BATCH, Action_BATCH, Reward_BATCH, State_Next_BATCH, non_final_mask = self.Memory.sample(BATCH_SIZE)
        Q_values = self.Network(State_BATCH).gather(1, Action_BATCH.reshape(-1,1)) # Action_BATCHのshapeは(-1)ではなく、(-1,1)の二次元である必要がある
        
        next_Q_values = torch.zeros(BATCH_SIZE) #全部0で初期化しておいて、non_finalの場所だけ計算する
        next_Q_values[non_final_mask] = self.Network(State_Next_BATCH).max(1)[0].detach()
        # max()メソッドで最大値を計算した後、[0]で数値を取り出す必要がある, detach()で勾配を消しておく    

        true_Q_values =  (Reward_BATCH + gamma * next_Q_values).unsqueeze(1)  #2次元配列に変換
        loss = F.smooth_l1_loss(Q_values, true_Q_values)

        self.optim.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optim.step()  # 結合パラメータを更新

In [207]:
AA = Agent()
BATCH_SIZE = 10
gamma = 0.99

for i in range(10):
    i = np.array(i)
    if i != 9:
        AA.Memory.put_memory( np.array((i,i,i,i)) ,np.array(0) ,i , np.array((i+1,i+1,i+1,i+1)) )
    else:
        AA.Memory.put_memory( np.array((i,i,i,i)), np.array(0), i, None )

AA.get_action([0,0,0,0], 1)

array(0)

In [110]:
Action_BATCH

tensor([0, 9, 6, 2, 4, 7, 1, 8, 5, 3])

# Task

In [8]:
Task_model = gym.make('CartPole-v0')

# Training Environment

In [23]:
class Environment:
    def __init__(self):
        self.Task  = Task_model
        self.Agent = Agent(4, 2)
    
    def run_one_episode(self, episode):
        # 倒れるか、200秒持ち堪え耐えるまでが１エピソード
        # 最初にモデルに今回のState_t, 前回のReword_t-1 を渡す。
        # AgentはState_tを元にV_t, pi_t を計算して記憶する
        
        t = 0
        state = self.Task.reset() # self.TaskからState_tを受け取る
        state = torch.from_numpy(state).type(torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
        state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換
        reword = 0

        while True:
            action  = self.Agent.get_action(state, 500)
            next_state, reword, done, info = self.Task.step(action.item()) #Action_tを場面に渡してState_t+1, Reword_tを受け取る
            next_state = torch.from_numpy(next_state).type(torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            next_state = torch.unsqueeze(next_state, 0)  # size 4をsize 1x4に変換
            
            if done: #倒れていたor終了した場合
                next_state = None
                if t < 195: #195ステップいないなら失敗
                    reword = torch.FloatTensor([-1.0])
                else: #195ステップ以上たってたら成功
                    reword = torch.FloatTensor([1.0])

            else: #各ステップで立ってたら報酬追加
                reword = torch.FloatTensor([0.0])
            
            self.Agent.memorize(state, action, next_state, reword)
            self.Agent.update_q_function()

            state = next_state
            t += 1

            if done:
                #print(self.Agent.V_values)
                #self.Agent.reset_model(episode)
                break
        return t
    
    def train(self):
        TIMES = []
        times = 0
        episode = 0
        while True:
            times = self.run_one_episode(episode)
            print('episode :', episode)
            print(times)
            print('='*10)
            episode += 1
            TIMES.append(times)

            if (len(TIMES) > 10) & (np.mean(TIMES[:-10]) > 150):
                
                break
        return TIMES

In [ ]:
GAMMA = 0.99
AAA = Environment()
TTT = AAA.train()

In [ ]:
X = range(len(TTT))
plt.plot(X, TTT)

In [ ]:
AAA.Agent.Next_States

In [37]:
len(AAA.Agent.States)

21767

# AAA

In [23]:
!pip install JSAnimation

from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0),
               dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                   interval=50)

    anim.save('movie_cartpole_DQN.mp4')  # 動画のファイル名と保存です
    display(display_animation(anim, default_mode='loop'))

  Created wheel for JSAnimation: filename=JSAnimation-0.1-cp37-none-any.whl size=11427 sha256=4f0fe318de78e3baabc6b72803f3f4fa69f982ba996d34d10963e0e10d1f40af
  Stored in directory: /root/.cache/pip/wheels/3c/c2/b2/b444dffc3eed9c78139288d301c4009a42c0dd061d3b62cead
Successfully built JSAnimation


In [24]:
# 定数の設定
ENV = 'CartPole-v0'  # 使用する課題名
GAMMA = 0.99  # 時間割引率
MAX_STEPS = 200  # 1試行のstep数
NUM_EPISODES = 500  # 最大試行回数



In [50]:
# namedtupleを生成
from collections import namedtuple

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

# 経験を保存するメモリクラスを定義します
class ReplayMemory:
    def __init__(self, CAPACITY):
        self.capacity = CAPACITY  # メモリの最大長さ
        self.memory = []  # 経験を保存する変数
        self.index = 0  # 保存するindexを示す変数

    def push(self, state, action, state_next, reward):
        '''transition = (state, action, state_next, reward)をメモリに保存する'''

        if len(self.memory) < self.capacity:
            self.memory.append(None)  # メモリが満タンでないときは足す

        self.memory[self.index] = Transition(state, action, state_next, reward)

        self.index = (self.index + 1) % self.capacity  # 保存するindexを1つずらす

    def sample(self, batch_size):
        '''batch_size分だけ、ランダムに保存内容を取り出す'''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''関数lenに対して、現在の変数memoryの長さを返す'''
        return len(self.memory)

# エージェントが持つ脳となるクラスです、DQNを実行します
# Q関数をディープラーニングのネットワークをクラスとして定義

import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000


class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)  # ネットワークの形を出力

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

    def replay(self):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        if len(self.memory) < BATCH_SIZE:
            return

        transitions = self.memory.sample(BATCH_SIZE)
        batch = Transition(*zip(*transitions))

        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])

        state_action_values = self.model(state_batch).gather(1, action_batch)

        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))
        # まずは全部0にしておく
        next_state_values = torch.zeros(BATCH_SIZE)
        next_state_values[non_final_mask] = self.model(non_final_next_states).max(1)[0].detach()

        expected_state_action_values = reward_batch + GAMMA * next_state_values

        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

        # 4.3 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optimizer.step()  # 結合パラメータを更新

    def decide_action(self, state, episode):
        '''現在の状態に応じて、行動を決定する'''
        # ε-greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値のindexを取り出します = max(1)[1]
            # .view(1,1)は[torch.LongTensor of size 1]　を size 1x1 に変換します

        else:
            # 0,1の行動をランダムに返す
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 0,1の行動をランダムに返す
            # actionは[torch.LongTensor of size 1x1]の形になります

        return action

In [47]:
class Agent:
    def __init__(self, num_states, num_actions):
        '''課題の状態と行動の数を設定する'''
        self.brain = Brain(num_states, num_actions)  # エージェントが行動を決定するための頭脳を生成

    def update_q_function(self):
        '''Q関数を更新する'''
        self.brain.replay()

    def get_action(self, state, episode):
        '''行動を決定する'''
        action = self.brain.decide_action(state, episode)
        return action

    def memorize(self, state, action, state_next, reward):
        '''memoryオブジェクトに、state, action, state_next, rewardの内容を保存する'''
        self.brain.memory.push(state, action, state_next, reward)

In [25]:
class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            observation = self.env.reset()  # 環境の初期化

            state = observation  # 観測をそのまま状態sとして使用
            state = torch.from_numpy(state).type(
                torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ

                if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
                    frames.append(self.env.render(mode='rgb_array'))

                action = self.agent.get_action(state, episode)  # 行動を求める

                # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
                # actionから.item()を指定して、中身を取り出す
                observation_next, _, done, _ = self.env.step(
                    action.item())  # rewardとinfoは使わないので_にする

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor(
                            [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                        complete_episodes = 0  # 連続成功記録をリセット
                    else:
                        reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                        complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor([0.0])  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                    state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Experience ReplayでQ関数を更新する
                self.agent.update_q_function()

                # 観測の更新
                state = state_next

                # 終了時の処理
                if done:
                    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                # 動画を保存と描画
                display_frames_as_gif(frames)
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする

In [93]:
# CartPoleを実行する環境のクラスです

class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent()  # 環境内で行動するAgentを生成

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            observation = self.env.reset()  # 環境の初期化

            state = observation  # 観測をそのまま状態sとして使用
            #state = torch.from_numpy(state).type(torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            #state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ
                action = self.agent.get_action(state)
                observation_next, _, done, _ = self.env.step(action)

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor( [-1.0] )  # 途中でこけたら罰則として報酬-1を与える
                        complete_episodes = 0  # 連続成功記録をリセット
                    else:
                        reward = torch.FloatTensor( [1.0] )  # 立ったまま終了時は報酬1を与える
                        complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor( [0.0] )  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    #state_next = torch.from_numpy(state_next).type(torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                    #state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.put_memory(state, action, reward, state_next)

                # Experience ReplayでQ関数を更新する
                self.agent.network_update()

                # 観測の更新
                state = state_next

                # 終了時の処理
                if done:
                    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする

In [106]:
class Agent:
    # 実際に行動を行うAgent
    # DQNで学習する
    def __init__(self):
        self.gamma = torch.tensor(0.99)
        self.episode = 0

        self.Memory = Agetnt_Memory()

        self.Network = Networks().double()
        self.Network_target = Networks().double()
        self.Network_target.load_state_dict( self.Network.state_dict() )

        torch.nn.utils.clip_grad_norm_(self.Network.parameters(), 0.1)
        self.optim = torch.optim.Adam(params=self.Network.parameters(), lr=0.0005 )
    
    def get_action(self, state):
        #ε-greedy法で行動を決定します
        #stateを入力として受け取り、actionを出力として返す
        #if self.episode >= 500:   
        #    eps = 0.0
        #else:
        #    eps = 0.3 * (1 - self.episode/500)  # linearly interpolate

        #if random.random() < eps:
        if 0 > 1:
            #action = random.randint(0, 2 - 1)   # ランダムに行動
            pass
        else:
            Q_values = self.Network( torch.tensor( state ).double() )
            p_values = nn.Softmax()( Q_values )
            action = np.random.choice(2, p=p_values.cpu().detach().numpy())
        return action

    def put_memory(self, state, action, reword, next_state):
        self.Memory.put_memory(state, action, reword, next_state)

    def network_update(self, BATCH_SIZE=BATCH_SIZE):
        # AgentのMemoriesからBATCH_SIZEの分だけデータをランダムに取り出す
        States_BATCH, Action_BATCH, Reword_BATCH, Next_States_BATCH, Next_S_non_mask = self.Memory.get_batch(BATCH_SIZE)

        Q_values = self.Network( States_BATCH.double() )
        Next_Q_values = torch.zeros_like( Q_values )

        Q_values = Q_values.gather( -1, Action_BATCH ) #Gatherメソッドでdim=1方向でそれぞれの行でスライスできる

        Next_Q_values[Next_S_non_mask] = self.Network_target( Next_States_BATCH.double() )
        Next_Q_values = Next_Q_values.max(dim=1).values

        True_Values = Reword_BATCH + self.gamma * Next_Q_values
        loss = nn.MSELoss()(Q_values, True_Values)
    
        self.optim.zero_grad()
        loss.backward()
        self.optim.step()
        #print('loss :', loss.cpu().detach())
    
    def reset_model(self, episode):
        self.episode = episode
        self.Network_target.load_state_dict( self.Network.state_dict() )

In [94]:
class Agetnt_Memory():
    def __init__(self):
        self.States   = []
        self.Actions = []
        self.Rewords = []
        self.Next_States = []
    
    def put_memory(self, state, action, reword, next_state):
        self.States.append( state )
        self.Actions.append(action)
        self.Rewords.append(reword)
        self.Next_States.append(next_state)

        if len(self.Actions) > 5000:
            self.States   = self.States[:-5000]
            self.Actions = self.Actions[:-5000]
            self.Rewords = self.Rewords[:-5000]
            self.Next_States = self.Next_States[:-5000]

    def get_batch(self, BATCH_SIZE):
        num_memories = len(self.States)
        sample_index = random.sample( range(num_memories), min([BATCH_SIZE, num_memories]) )

        States_BATCH = torch.tensor( np.stack( np.array(self.States)[sample_index]  ) )
        Action_BATCH = torch.tensor( np.stack( np.array(self.Actions)[sample_index] ) ).reshape(-1,1)
        Reword_BATCH = torch.tensor( np.stack( np.array(self.Rewords)[sample_index] ) )
        #Next_States_BATCH = torch.tensor( np.stack( np.array(self.Next_States)[sample_index] ) ).to(device)
        Next_States_BATCH = np.array(self.Next_States)[sample_index]
        Next_S_non_mask = [s is not None for s in Next_States_BATCH]
        Next_States_BATCH = torch.tensor( np.stack(Next_States_BATCH[Next_S_non_mask]) )
        return States_BATCH, \
               Action_BATCH, \
               Reword_BATCH, \
               Next_States_BATCH, \
               Next_S_non_mask

In [ ]:
Q_values = torch.tensor([[[0.0395, 0.0723]]])
A = torch.tensor([[1]])

Q_values[0].gather( -1, A )

In [107]:
# main クラス
cartpole_env = Environment()
cartpole_env.run()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([3])) that is di

0 Episode: Finished after 15 steps：10試行の平均step数 = 1.5
1 Episode: Finished after 14 steps：10試行の平均step数 = 2.9
2 Episode: Finished after 15 steps：10試行の平均step数 = 4.4
3 Episode: Finished after 27 steps：10試行の平均step数 = 7.1
4 Episode: Finished after 29 steps：10試行の平均step数 = 10.0
5 Episode: Finished after 13 steps：10試行の平均step数 = 11.3
6 Episode: Finished after 12 steps：10試行の平均step数 = 12.5
7 Episode: Finished after 30 steps：10試行の平均step数 = 15.5
8 Episode: Finished after 8 steps：10試行の平均step数 = 16.3
9 Episode: Finished after 13 steps：10試行の平均step数 = 17.6
10 Episode: Finished after 22 steps：10試行の平均step数 = 18.3
11 Episode: Finished after 49 steps：10試行の平均step数 = 21.8
12 Episode: Finished after 38 steps：10試行の平均step数 = 24.1
13 Episode: Finished after 12 steps：10試行の平均step数 = 22.6
14 Episode: Finished after 11 steps：10試行の平均step数 = 20.8
15 Episode: Finished after 20 steps：10試行の平均step数 = 21.5
16 Episode: Finished after 21 steps：10試行の平均step数 = 22.4
17 Episode: Finished after 27 steps：10試行の平均step数 = 22.1
18 Epis

# BBB

In [39]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

class Memories():
    def __init__(self):
        self.memory = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.memory.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.memory, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), \
               torch.tensor(a_lst), \
               torch.tensor(r_lst), \
               torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.memory)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main():
    env = gym.make('CartPole-v1')
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = Memories()

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False

        while not done:
            a = q.sample_action( torch.from_numpy(s).float(), epsilon )      
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put( (s,a,r/100.0,s_prime, done_mask) )
            s = s_prime

            score += r
            if done:
                break
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, memory_size : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()

n_episode :20, score : 10.0, memory_size : 200, eps : 7.9%
n_episode :40, score : 9.6, memory_size : 392, eps : 7.8%
n_episode :60, score : 9.8, memory_size : 587, eps : 7.7%
n_episode :80, score : 9.4, memory_size : 775, eps : 7.6%
n_episode :100, score : 9.7, memory_size : 968, eps : 7.5%
n_episode :120, score : 9.7, memory_size : 1162, eps : 7.4%
n_episode :140, score : 10.1, memory_size : 1363, eps : 7.3%
n_episode :160, score : 9.7, memory_size : 1557, eps : 7.2%
n_episode :180, score : 9.8, memory_size : 1752, eps : 7.1%
n_episode :200, score : 9.9, memory_size : 1951, eps : 7.0%
n_episode :220, score : 10.2, memory_size : 2156, eps : 6.9%
n_episode :240, score : 10.1, memory_size : 2358, eps : 6.8%
n_episode :260, score : 14.4, memory_size : 2647, eps : 6.7%
n_episode :280, score : 17.9, memory_size : 3005, eps : 6.6%
n_episode :300, score : 48.4, memory_size : 3972, eps : 6.5%
n_episode :320, score : 73.4, memory_size : 5440, eps : 6.4%
n_episode :340, score : 108.6, memory_siz

KeyboardInterrupt: ignored